In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-cohere to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━

In [2]:
!pip install 'crewai[tools]'

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.3/463.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=787c3a575debfb0e3ca080e385bafe06d065b626331e96d6c723df9511fb2d37
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9

In [3]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

os.environ["OPENAI_API_KEY"] = "ENTER YOUR OPEN AI API"
os.environ["SERPER_API_KEY"] = "ENTER YOUR SERPER API" # serper.dev API key

# You can choose to use a local model through Ollama for example. See https://docs.crewai.com/how-to/LLM-Connections/ for more information.

# os.environ["OPENAI_API_BASE"] = 'http://localhost:11434/v1'
# os.environ["OPENAI_MODEL_NAME"] ='openhermes'  # Adjust based on available model
# os.environ["OPENAI_API_KEY"] ='sk-111111111111111111111111111111111111111111111111'

# You can pass an optional llm attribute specifying what model you wanna use.
# It can be a local model through Ollama / LM Studio or a remote
# model like OpenAI, Mistral, Antrophic or others (https://docs.crewai.com/how-to/LLM-Connections/)
# If you don't specify a model, the default is OpenAI gpt-4o
#
# import os
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo'
#
# OR
#
# from langchain_openai import ChatOpenAI

search_tool = SerperDevTool()

# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  # You can pass an optional llm attribute specifying what model you wanna use.
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7),
  tools=[search_tool]
)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True
)

# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=True,
  process = Process.sequential
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

 [2024-09-14 08:09:11][DEBUG]: == Working Agent: Senior Research Analyst
 [2024-09-14 08:09:11][INFO]: == Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


> Entering new CrewAgentExecutor chain...
I need to gather comprehensive information on the latest advancements in AI in 2024, including key trends, breakthrough technologies, and potential industry impacts. To do this, I will conduct an internet search to find the most recent and relevant information available.

Action: Search the internet
Action Input: {"search_query": "latest advancements in AI 2024"} 


Search results: Title: 7 rapid AI trends happening in 2024 - Khoros
Link: https://khoros.com/blog/ai-trends
Snippet: 7 rapid AI trends happening in 2024 · 1. Multimodal AI · 2. Small language models · 3. Customizable generative AI · 4. New use cases · 5. Shadow ...
---
Title: Biggest AI Developments of 2024 

In [10]:
# Step 1: Install Required Libraries
!pip install langchain openai duckduckgo-search

# Step 2: Import Required Libraries
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun
import os

# Step 3: Set your OpenAI API key
# Replace "your-openai-api-key-here" with your actual OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-9p_fP6686InNCCoRJLVFb3qGWSSvwKEeolXmZ1e-noT3BlbkFJyWG1CMLPitnCugDqcfWHfKbfcCtxz7F50Ep3Zy7G0A"

# Step 4: Load GPT-3.5 Model
llm = ChatOpenAI(model='gpt-3.5-turbo')

# Step 5: Define DuckDuckGo search tool for real-time search capability
search_tool = Tool(
    name="Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to search for current information on the web"
)

# Step 6: Define roles (local expert, historian, food expert) for the crew
# Each role will act as an "agent" with specific responsibilities

# Define a local expert agent
def local_expert():
    agent = initialize_agent(
        tools=[search_tool],
        llm=llm,
        agent="zero-shot-react-description",
        verbose=True
    )
    return agent

# Define a historian agent for historical insights
def historian():
    agent = initialize_agent(
        tools=[search_tool],
        llm=llm,
        agent="zero-shot-react-description",
        verbose=True
    )
    return agent

# Define a food expert agent to recommend food spots
def food_expert():
    agent = initialize_agent(
        tools=[search_tool],
        llm=llm,
        agent="zero-shot-react-description",
        verbose=True
    )
    return agent

# Step 7: Run Crew AI Agents
# The agents will now work together to provide insights about a selected city
def crew_ai(city):
    # Local Expert
    local_agent = local_expert()
    local_info = local_agent.run(f"What are the top tourist attractions in {city}?")

    # Historian
    history_agent = historian()
    historical_info = history_agent.run(f"What is the historical significance of {city}?")

    # Food Expert
    food_agent = food_expert()
    food_info = food_agent.run(f"What are the best restaurants to try in {city}?")

    # Combining the results from all agents (Crew AI collaboration)
    result = f"""Here is a detailed plan for {city}:

    **Top Attractions:**
    {local_info}

    **Historical Insights:**
    {historical_info}

    **Food Recommendations:**
    {food_info}
    """

    return result

# Step 8: Ask the Crew AI about a city
city = "Paris"
crew_response = crew_ai(city)
print(crew_response)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 28.6 MB/s eta 0:00:00


<ipython-input-10-a9f62e255453>:29: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(
<ipython-input-10-a9f62e255453>:62: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  local_info = local_agent.run(f"What are the top tourist attractions in {city}?")




> Entering new AgentExecutor chain...
I should search for a list of the top tourist attractions in Paris to provide an accurate answer.
Action: Search
Action Input: "top tourist attractions in Paris"
Observation: Discover the best places to visit in Paris, from the iconic Eiffel Tower and Notre-Dame Cathedral to the charming Montmartre and Luxembourg Gardens. Learn about the history, culture, and cuisine of the City of Light with our guide and tips. The Montparnasse Tower Observation Deck claims to have the best views in Paris - and once you reach the top, it's easy to see why. The lower deck stands more than 650 feet high and overlooks ... Arc de Triomphe. The Arc de Triomphe was commissioned by Napoleon in 1806 to celebrate his victory at the Battle of Austerlitz. Climb the nearly 300 steps or take the elevator to the top for some ... Along the way, pop into Laduree for macarons, another must do while in Paris. The Champs-Elysees and the Arc de Triomphe | Best Things to Do in Paris